In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd "/content/drive/MyDrive/hotdog-nothotdog"
!ls

/content/drive/MyDrive/hotdog-nothotdog
augmented  hotdog.jpg  pizza.jpg  test	train  validation


In [3]:
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.layers import BatchNormalization, Dropout, Dense
from tensorflow.keras.callbacks import EarlyStopping
import os

base_dir = '/content/drive/MyDrive/hotdog-nothotdog'
test_dir = os.path.join(base_dir, 'test')
validation_dir = os.path.join(base_dir, 'validation')
train_dir = os.path.join(base_dir, 'train')


In [4]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    validation_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary'
)

test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(224, 224),
    batch_size=32,
    label_mode='binary'
)


Found 4042 files belonging to 2 classes.
Found 200 files belonging to 2 classes.
Found 400 files belonging to 2 classes.


In [5]:
import tensorflow_hub as hub

model = models.Sequential()
model_E = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/mobilenet_v2_100_224/classification/5",
                   trainable=False),
    tf.keras.layers.Dense(128, activation='relu')
])

model.add(model_E)
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(50, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.build((None, 224, 224, 3))
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 128)               3668521   
                                                                 
 batch_normalization (Batch  (None, 128)               512       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 50)                6450      
                                                                 
 dense_2 (Dense)             (None, 1)                 51        
                                                                 
Total params: 3675534 (14.02 MB)
Trainable params: 135013 (527.39 KB)
Non-trainable params: 3540521 (13.51 MB)
___________

In [6]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(
    train_dataset,
    epochs=20,
    validation_data=validation_dataset,
    callbacks=[early_stopping]
)


Epoch 1/20
127/127 [==============================] - 671s 5s/step - loss: 0.6074 - accuracy: 0.6739 - val_loss: 0.7209 - val_accuracy: 0.6250
Epoch 2/20
127/127 [==============================] - 160s 1s/step - loss: 0.5524 - accuracy: 0.7118 - val_loss: 0.8218 - val_accuracy: 0.5900
Epoch 3/20
127/127 [==============================] - 156s 1s/step - loss: 0.5208 - accuracy: 0.7380 - val_loss: 0.7324 - val_accuracy: 0.6250
Epoch 4/20
127/127 [==============================] - 169s 1s/step - loss: 0.4935 - accuracy: 0.7536 - val_loss: 0.7002 - val_accuracy: 0.6750
Epoch 5/20
127/127 [==============================] - 165s 1s/step - loss: 0.4823 - accuracy: 0.7610 - val_loss: 0.6842 - val_accuracy: 0.6500
Epoch 6/20
127/127 [==============================] - 147s 1s/step - loss: 0.4738 - accuracy: 0.7677 - val_loss: 0.6975 - val_accuracy: 0.6400
Epoch 7/20
127/127 [==============================] - 142s 1s/step - loss: 0.4550 - accuracy: 0.7761 - val_loss: 0.7217 - val_accuracy: 0.6850

In [ ]:
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['loss'], label='train loss')
plt.plot(history.history['val_loss'], label='val loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.show()
